In [1]:


from SBLMDCOVDOCK.SBLSettings import DOCKING_Settings
from SBLMDCOVDOCK.md_setuptools import plot_xvg
import pickle
from copy import deepcopy
import subprocess

import pandas as pd
import os
import glob

# import gmxapi as gmx
import subprocess
import matplotlib.pyplot as plt

settings = DOCKING_Settings()
main_dir = os.getcwd()
import shutil

# AMPC - https://journals.asm.org/doi/10.1128/AAC.02073-20
ampc = "6T3D"
# KPC-2 - BLDB: http://dx.doi.org/10.1021/ACS.JMEDCHEM.7B00158
kpc2 = "5UL8"
# OXA-10 - BLDB: https://www.pnas.org/doi/full/10.1073/pnas.241442898
oxa10 = "1K55"

structures = pd.DataFrame({"PDBID": [ampc, kpc2, oxa10], 
                           "Name": ["AmpC", "KPC2", "OXA10"],
                           "Cat_Ser": [64, 70, 67],
                           "start": [4, 23, 21]})
                           

dir = "/home/alexi/Documents/SBL_MD_CovDock/topology/Dock"
pythonsh= "/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/bin/pythonsh"


In [2]:
#2. Generate PDBQT files for protein
# break
pythonsh= "/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/bin/pythonsh"
print(dir)
for pdb in structures.PDBID:
    break
    script_name = 'prepare_receptor4.py'
    prepare_rec_path = os.path.join("..","..",os.path.join(settings.config, settings.parent, script_name))
    pdb_file =  "APO_" +pdb + "_npt" + ".pdb"
    # pdb_file =  "APO_" +pdb + "_chimeraFH" + ".pdb"

    pdb_path = os.path.join(dir, pdb_file)
    cat_ser = str(structures[structures.PDBID == pdb].Cat_Ser.values[0])
    start = str(structures[structures.PDBID == pdb].start.values[0])
    out_pdb_name = "APO_" +pdb + "_ADT" + ".pdb" # extract from MD

    out_pdb_path = os.path.join(dir, out_pdb_name)
    # renumber residues
    !pdb_reres -$start $pdb_path | pdb_chain -A > $out_pdb_path


    pdbqt_name = out_pdb_name.replace(".pdb", ".pdbqt")


    prepare_receptor_command = [pythonsh, prepare_rec_path,
                                "-r", out_pdb_path,
                                "-o", pdbqt_name,
                                "-A", "bonds_hydrogens",
                                "-U", "nphs_lps",
                                "-e", 
                                "-v",

                                ]

    print(" ".join(prepare_receptor_command))
    subprocess.run(prepare_receptor_command, cwd = dir,check=True)

    break

/home/alexi/Documents/SBL_MD_CovDock/topology/Dock


In [3]:
import os
import subprocess

sdf_complex_dir = os.path.join(settings.structures_output, "Ligand_Complex")
for idx, row in structures.iterrows():
    break
    pdb = row["PDBID"]
    cat_ser = row["Cat_Ser"]
    pdb_file =  "APO_" +pdb + "_ADT" + ".pdb" # extract from MD
    pdb_file = os.path.join(dir, pdb_file)
    ser_pdb =  "APO_" +pdb + "_ADT" + "_SER"+".pdb" # extract from MD
    ser_out = os.path.join(dir, ser_pdb)
    print(pdb_file, ser_pdb)

    # extract the serine from the receptor using subprocess

    !pdb_selres -$cat_ser $pdb_file | pdb_reatom | pdb_tidy > $ser_out

    
    sdf_out = ser_pdb.replace(".pdb", ".sdf")
    # convert the serine to SDF and protonate it using subprocess
    subprocess.run(["obabel", ser_pdb, "-O", sdf_out], cwd=dir,check=True)


In [4]:
sdf_complex_dir = os.path.join(settings.structures_output, "Ligand_Complex")
sdf_complex_out = os.path.join(settings.structures_output, "Complex_Aligned")
# break
os.makedirs(sdf_complex_dir, exist_ok=True)
os.makedirs(sdf_complex_out, exist_ok=True)
# perform mcs alignment for each pdb and ligand
# need to extract coordinates and then apply these to the ligand files
for pdb in structures.PDBID:
    pdb = "6T3D"
    break
    for lig in os.listdir(sdf_complex_dir):
        print(lig)
        lig_sdf = lig 
        lig = lig.replace(".sdf", "").split("_")[0]

        sdf_out = f"{pdb}_SER_{lig}.sdf"
        template_ser_sdf = "APO_" +pdb + "_ADT" + "_SER"+".sdf" # extract from MD
        template_ser_sdf = os.path.join(settings.topology, settings.parent, template_ser_sdf)  

        lig_sdf = os.path.join(sdf_complex_dir, lig_sdf)
        sdf_out = os.path.join(sdf_complex_out, sdf_out)

        script_name = "mcs_constrained_alignment_SER.py"
        mcs_script_path = os.path.join(settings.config, settings.parent, script_name)
        mcs_command = [mcs_script_path,
                       template_ser_sdf,
                       lig_sdf,
                       sdf_out,
                       str(1), str(2), str(1), # default is 100??? # we only want one conformation
                       str(0)]
        
        print(" ".join(mcs_command))
        subprocess.run(mcs_command)
        # break
    break

In [5]:
# convert comlplex SDF to PDB 
# change this to apply the coordinates from the aligned sdf files to the pdb version of the ligand
# sdf_complex_dir = os.path.join(settings.structures_output, "Ligand_Complex")
sdf_complex_out = os.path.join(settings.structures_output, "Complex_Aligned")
pdb_complex_out = os.path.join(settings.structures_output, "Aligned_PDB")
re_res_complex = os.path.join(settings.structures_output, "Complex_ReRes")
# break
os.makedirs(pdb_complex_out, exist_ok=True)
os.makedirs(re_res_complex, exist_ok=True)
# Convert sdf to pdb
# for pdb in structures.PDBID:
pdb = "6T3D"
for lig in os.listdir(sdf_complex_out):
    break
    print(lig)

    pdb_name = lig.replace(".sdf", ".pdb")
    pdb_path = os.path.join(pdb_complex_in, pdb_name)

    sdf_path = os.path.join(sdf_complex_out, lig)
    # open sdf and extract coordinates


    # open pdb

    # replace pdb coordinates with those from sdf

    # save pdb to re_res_complex
    


    


In [6]:
import os
pdb_complex_in = os.path.join(settings.structures_input, "Ligand_Complex_Stripped")

# Assuming settings and other directory paths are defined above

# Function to extract coordinates from SDF file
def extract_coordinates_from_sdf(sdf_path):
    coords = []
    with open(sdf_path, 'r') as f:
        lines = f.readlines()
        atom_count = int(lines[3].strip().split()[0])
        for line in lines[4:4+atom_count]:
            x, y, z = line.split()[0:3]
            coords.append((x, y, z))
    return coords

# Function to replace coordinates in PDB file
def replace_coordinates_in_pdb(pdb_path, new_coords):
    new_content = []
    with open(pdb_path, 'r') as f:
        lines = f.readlines()
        coord_idx = 0
        for line in lines:
            if line.startswith("ATOM") or line.startswith("HETATM"):
                x, y, z = new_coords[coord_idx]
                coord_idx += 1
                new_line = f"{line[:30]}{float(x):8.3f}{float(y):8.3f}{float(z):8.3f}{line[54:]}"
                new_content.append(new_line)
            else:
                new_content.append(line)
    return new_content

# Loop through the ligands in the sdf_complex_out directory
for lig in os.listdir(sdf_complex_out):
    print(lig)
    ligcode = lig.split("_")[-1].replace(".sdf", "")
    print(ligcode)

    pdb_name = [x for x in os.listdir(pdb_complex_in) if ligcode in x][0]
    print(pdb_name)
    # print(sorted(os.listdir(pdb_complex_in)))
    pdb_path = os.path.join(pdb_complex_in, pdb_name)
    output_pdb_path = os.path.join(re_res_complex, lig.replace(".sdf", ".pdb"))
    sdf_path = os.path.join(sdf_complex_out, lig)
    # print(sorted(os.listdir(sdf_complex_out)))

    # Extract coordinates from SDF
    try:
        print("Extracting coordinates from SDF: ", lig)
        new_coords = extract_coordinates_from_sdf(sdf_path)
        try:
            print("Replacing coordinates in PDB: ", pdb_name)
            new_pdb_content = replace_coordinates_in_pdb(pdb_path, new_coords)
        except:
            print("Failed to replace coordinates in PDB")
            continue
        
    except:
        print("Failed to extract coordinates from SDF")
        continue

        # Replace coordinates in PDB
        
    # Save modified PDB to re_res_complex directory
    with open(output_pdb_path, 'w') as f:
        f.writelines(new_pdb_content)


6T3D_SER_0NB.sdf
0NB
0NB_4E3L.pdb
Extracting coordinates from SDF:  6T3D_SER_0NB.sdf
Replacing coordinates in PDB:  0NB_4E3L.pdb
6T3D_SER_AXL.sdf
AXL
AXL_3N7W.pdb
Extracting coordinates from SDF:  6T3D_SER_AXL.sdf
Replacing coordinates in PDB:  AXL_3N7W.pdb
6T3D_SER_DRW.sdf
DRW
DRW_3IQA.pdb
Extracting coordinates from SDF:  6T3D_SER_DRW.sdf
Replacing coordinates in PDB:  DRW_3IQA.pdb
6T3D_SER_SA2.sdf
SA2
SA2_2H5S.pdb
Extracting coordinates from SDF:  6T3D_SER_SA2.sdf
Failed to extract coordinates from SDF
6T3D_SER_CZ8.sdf
CZ8
CZ8_3MXR.pdb
Extracting coordinates from SDF:  6T3D_SER_CZ8.sdf
Replacing coordinates in PDB:  CZ8_3MXR.pdb
6T3D_SER_LOR.sdf
LOR
LOR_1FCN.pdb
Extracting coordinates from SDF:  6T3D_SER_LOR.sdf
Replacing coordinates in PDB:  LOR_1FCN.pdb
6T3D_SER_NFF.sdf
NFF
NFF_3NCK.pdb
Extracting coordinates from SDF:  6T3D_SER_NFF.sdf
Replacing coordinates in PDB:  NFF_3NCK.pdb
6T3D_SER_8CY.sdf
8CY
8CY_5UJO.pdb
Extracting coordinates from SDF:  6T3D_SER_8CY.sdf
Replacing coordin

In [11]:
# re res all entries to 64
pdb = "6T3D"
cat_ser = str(structures[structures.PDBID == pdb].Cat_Ser.values[0])

for lig in os.listdir(re_res_complex):
    # break
    lig_path = os.path.join(re_res_complex, lig)

    with open(lig_path, "r+") as f:
        lines = f.readlines()

        new_lines = []
        for line in lines:
            if line.startswith("ATOM") or line.startswith("HETATM"):
                print(line)
                # replace resname
                # line = line[:17] + "SER" + line[20:]
                #replace HETATM
                # line = line[:0] + "ATOM  " + line[6:]
                # replace resnum
                line = line[:24] + cat_ser + line[26:] ### check this
                print(line)
                
            new_lines.append(line)

    with open(lig_path, "w+") as f:
        f.writelines(new_lines)
    print(lig_path)

ATOM     13  N   SER A  64      48.800  51.874  46.337  1.00 25.41      A    N

ATOM     13  N   SER A  64      48.800  51.874  46.337  1.00 25.41      A    N

ATOM     14  CA  SER A  64      47.448  51.383  46.543  1.00 27.10      A    C

ATOM     14  CA  SER A  64      47.448  51.383  46.543  1.00 27.10      A    C

ATOM     15  C   SER A  64      47.404  49.888  46.440  1.00 32.44      A    C

ATOM     15  C   SER A  64      47.404  49.888  46.440  1.00 32.44      A    C

ATOM     16  O   SER A  64      46.474  49.270  46.937  1.00 26.93      A    O

ATOM     16  O   SER A  64      46.474  49.270  46.937  1.00 26.93      A    O

ATOM     17  CB  SER A  64      46.516  52.003  45.509  1.00 31.95      A    C

ATOM     17  CB  SER A  64      46.516  52.003  45.509  1.00 31.95      A    C

ATOM     18  OG  SER A  64      46.159  53.332  45.873  1.00 30.70      A    O

ATOM     18  OG  SER A  64      46.159  53.332  45.873  1.00 30.70      A    O

HETATM   85  CAK DGF A  64      43.731  

In [12]:
print(len(os.listdir(re_res_complex)))

168


In [9]:
import os

# Assuming you have a DataFrame named "structures" containing the necessary data
# and "pdbqt_complex_out" is the directory containing PDB files

pdb = "6T3D"
resn = "SER"

# Assuming structures DataFrame has a column named "PDBID" and "Cat_Ser"
cat_ser = structures[structures.PDBID == pdb].Cat_Ser.values[0]

for lig in os.listdir(pdb_complex_out):
    break
    if lig.endswith('.pdb'):  # Make sure to process only PDB files
        print(lig)
        pdb_path = os.path.join(pdb_complex_out, lig)

        with open(pdb_path, "r+") as f:
            lines = f.readlines()

            new_lines = []
            new_residue_index = cat_ser  # Starting index for the modified residue numbers

            for line in lines:
                # print(line)
                if line.startswith("ATOM") or line.startswith("HETATM"):
                    atom_serial = int(line[6:11])
                    residue_index = int(line[22:26])
                    old_resname = line[17:20]
                    print(atom_serial, residue_index, old_resname)
                    print(line)
                    # Update residue name and residue number
                    if old_resname != resn and residue_index != cat_ser:
                        line = line[:17] + resn + line[20:]
                        line = line[:22] + str(new_residue_index).rjust(4) + line[26:]
                    print(line)   
                    new_lines.append(line)

            # Write modified lines back to the file
            f.seek(0)
            f.writelines(new_lines)
            f.truncate()
        # break

print("Residue renaming and numbering changes completed.")


Residue renaming and numbering changes completed.


In [23]:
#2. Generate PDBQT files for the ligands
pdbqt_complex_out = os.path.join(settings.structures_output, "Aligned_PDBQT")
# ligand_complex_stripped = os.path.join(settings.structures_input, "Ligand_Complex_Stripped")

# break
os.makedirs(pdbqt_complex_out, exist_ok=True)

script_name = 'prepare_receptor4.py'
prepare_rec_path = os.path.join(os.path.join(settings.config, settings.parent, script_name))
pdb = "6T3D"
cat_ser = structures[structures.PDBID == pdb].Cat_Ser.values[0]

for lig in os.listdir(re_res_complex):
    # break

    lig_path = os.path.join(re_res_complex, lig)

    with open(lig_path, "r+") as f:
        lines = f.readlines()

        new_lines = []
        for line in lines:
            if line.startswith("ATOM") or line.startswith("HETATM"):
                print(line)
                # replace resname
                line = line[:17] + "SER" + line[20:]
                # replace resnum
                line = line[:24] + str(cat_ser) + line[26:]
                #replace HETATM
                # line = line[:0] + "ATOM  " + line[6:]
                print(line)
                
            new_lines.append(line)
    # continue
    with open(lig_path, "w+") as f:
        f.writelines(new_lines)
    print(lig_path)

    pdb_path = os.path.join(re_res_complex, lig)
    pdbqt_path = os.path.join(pdbqt_complex_out, lig.replace(".pdb", ".pdbqt"))

    prepapre_cov_lig_command = [pythonsh, prepare_rec_path,
                                "-r", pdb_path,
                                "-o", pdbqt_path,
                                "-A", "bonds_hydrogens",
                                "-v"]
    
    print(" ".join(prepapre_cov_lig_command))
    subprocess.run(prepapre_cov_lig_command)



ATOM     13  N   SER A  67      48.800  51.874  46.337  1.00 25.41      A    N

ATOM     13  N   SER A  64      48.800  51.874  46.337  1.00 25.41      A    N

ATOM     14  CA  SER A  67      47.448  51.383  46.543  1.00 27.10      A    C

ATOM     14  CA  SER A  64      47.448  51.383  46.543  1.00 27.10      A    C

ATOM     15  C   SER A  67      47.404  49.888  46.440  1.00 32.44      A    C

ATOM     15  C   SER A  64      47.404  49.888  46.440  1.00 32.44      A    C

ATOM     16  O   SER A  67      46.474  49.270  46.937  1.00 26.93      A    O

ATOM     16  O   SER A  64      46.474  49.270  46.937  1.00 26.93      A    O

ATOM     17  CB  SER A  67      46.516  52.003  45.509  1.00 31.95      A    C

ATOM     17  CB  SER A  64      46.516  52.003  45.509  1.00 31.95      A    C

ATOM     18  OG  SER A  67      46.159  53.332  45.873  1.00 30.70      A    O

ATOM     18  OG  SER A  64      46.159  53.332  45.873  1.00 30.70      A    O

HETATM   85  CAK SER A  67      43.731  

Traceback (most recent call last):
  File "config/Dock/prepare_receptor4.py", line 172, in <module>
    delete_single_nonstd_residues=delete_single_nonstd_residues)    
  File "/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/CCSBpckgs/AutoDockTools/MoleculePreparation.py", line 558, in __init__
    version=version, delete_single_nonstd_residues=delete_single_nonstd_residues)
  File "/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/CCSBpckgs/AutoDockTools/MoleculePreparation.py", line 124, in __init__
    self.repairMol(mol, self.repair_type_list)
  File "/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/CCSBpckgs/AutoDockTools/MoleculePreparation.py", line 174, in repairMol
    self.newHs = self.addHydrogens(mol)
  File "/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/CCSBpckgs/AutoDockTools/MoleculePreparation.py", line 187, in addHydrogens
    HB.addHydrogens(mol)
  File "/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/CCSBpckgs/MolKit/hydrogenBuilder.py", line 61, in addHydrogens
    b

ATOM     25  N   SER A  67      48.796  51.877  46.337  1.00 15.50      A    N

ATOM     25  N   SER A  64      48.796  51.877  46.337  1.00 15.50      A    N

ATOM     26  CA  SER A  67      47.444  51.386  46.542  1.00 18.06      A    C

ATOM     26  CA  SER A  64      47.444  51.386  46.542  1.00 18.06      A    C

ATOM     27  C   SER A  67      47.401  49.890  46.437  1.00 17.54      A    C

ATOM     27  C   SER A  64      47.401  49.890  46.437  1.00 17.54      A    C

ATOM     28  O   SER A  67      46.475  49.270  46.940  1.00 16.81      A    O

ATOM     28  O   SER A  64      46.475  49.270  46.940  1.00 16.81      A    O

ATOM     29  CB  SER A  67      46.516  52.011  45.507  1.00 18.63      A    C

ATOM     29  CB  SER A  64      46.516  52.011  45.507  1.00 18.63      A    C

ATOM     30  OG  SER A  67      46.168  53.316  45.877  1.00 19.27      A    O

ATOM     30  OG  SER A  64      46.168  53.316  45.877  1.00 19.27      A    O

HETATM   73  C15 SER A  67      44.806  

In [24]:
print(len(os.listdir(pdbqt_complex_out)))

167


In [25]:
#3. Generate flexible/rigid PDBQT files for both the ligands and the receptors
flex_out = os.path.join(settings.structures_output, "PDBQT_Flex")
rigid_out = os.path.join(settings.structures_output, "PDBQT_Rigid")

os.makedirs(flex_out, exist_ok=True)
os.makedirs(rigid_out, exist_ok=True)

# 3.1 Generate flex and rigid PDBQT files for the ligands
script_name = 'prepare_flexreceptor4.py'

prepare_flex_path = os.path.join(os.path.join(settings.config, settings.parent, script_name))


pdb = "6T3D"
for lig in os.listdir(pdbqt_complex_out):
    # break
    cat_ser = structures[structures.PDBID == pdb].Cat_Ser.values[0]


    lig_path = os.path.join(pdbqt_complex_out, lig)

    flex_out_path = os.path.join(flex_out, lig)

    prepare_flex_lig_command = [pythonsh, prepare_flex_path,
                                "-r", lig_path,
                                "-s", "SER"+str(cat_ser),
                                "-g", flex_out_path.replace(".pdbqt", "_rigid.pdbqt").replace("PDBQT_Flex", "PDBQT_Rigid"),
                                "-x", flex_out_path.replace(".pdbqt", "_flex.pdbqt"),
                                "-v"]


    print(" ".join(prepare_flex_lig_command))
    try:
        subprocess.run(prepare_flex_lig_command, check=True)
    except:
        print("Error in", lig)
        pass



/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/bin/pythonsh config/Dock/prepare_flexreceptor4.py -r prod_structures/Ligand_Substrate/Aligned_PDBQT/6T3D_SER_APB.pdbqt -s SER64 -g prod_structures/Ligand_Substrate/PDBQT_Rigid/6T3D_SER_APB_rigid.pdbqt -x prod_structures/Ligand_Substrate/PDBQT_Flex/6T3D_SER_APB_flex.pdbqt -v


set verbose to True
read  prod_structures/Ligand_Substrate/Aligned_PDBQT/6T3D_SER_APB.pdbqt
Specified flexres selection strings are:
   SER64
selecting  SER64
res_names= ['SER64']
looking for   SER64
 added  ['SER64']  to  [<Residue instance> 6T3D_SER_APB:A:SER64 with 19 MolKit.molecule.Atom]
located  1  residues to format:
   6T3D_SER_APB:A:SER64
beginning formatting ...
finished!
/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/bin/pythonsh config/Dock/prepare_flexreceptor4.py -r prod_structures/Ligand_Substrate/Aligned_PDBQT/6T3D_SER_M9W.pdbqt -s SER64 -g prod_structures/Ligand_Substrate/PDBQT_Rigid/6T3D_SER_M9W_rigid.pdbqt -x prod_structures/Ligand_Substrate/PDBQT_Flex/6T3D_SER_M9W_flex.pdbqt -v
set verbose to True
read  prod_structures/Ligand_Substrate/Aligned_PDBQT/6T3D_SER_M9W.pdbqt
Specified flexres selection strings are:
   SER64
selecting  SER64
res_names= ['SER64']
looking for   SER64
 added  ['SER64']  to  [<Residue instance> 6T3D_SER_M9W:A:SER64 with 31 MolKit.molecule.Atom

In [27]:
print(len(os.listdir(flex_out)))

167


In [28]:

# break
# 3.2 Generate flex and rigid PDBQT files for the receptors

for pdb in structures.PDBID:
    break
    pdb = "6T3D"
    cat_ser = structures[structures.PDBID == pdb].Cat_Ser.values[0]

    # pdb_file = f"APO_{pdb}_chimeraFH.pdb"

    out_pdb_name = "APO_" +pdb + "_ADT" + ".pdb"
    pdbqt_name = out_pdb_name.replace(".pdb", ".pdbqt")

    pdbqt_path = os.path.join(dir, pdbqt_name)

    prepare_flex_rec_command = [pythonsh, prepare_flex_path,
                                "-r", pdbqt_path,
                                "-s", "SER"+str(cat_ser),
                                "-g", pdbqt_path.replace(".pdbqt", "_rigid.pdbqt"),
                                "-x", pdbqt_path.replace(".pdbqt", "_flex.pdbqt"),
                                "-v"]

    print(" ".join(prepare_flex_rec_command))
    subprocess.run(prepare_flex_rec_command,check=True)
    break

In [18]:
for pdb in structures.PDBID:
    # break
    # pdb_name = "APO_" +pdb + "_ADT_flex" + ".pdbqt"

    # orig_path = os.path.join(dir, pdb_name)

    # dest_path = os.path.join(flex_out, pdb_name)
    # print(os.getcwd())
    # # copy files to the flex_out directory
    # shutil.copy(orig_path, dest_path)
    # print(orig_path, dest_path)
    pdb_name = "APO_" +pdb + "_ADT_rigid" + ".pdbqt"
    orig_path = os.path.join(dir, pdb_name)

    dest_path = os.path.join(flex_out, pdb_name)
    shutil.copy(orig_path, dest_path)
    print(orig_path, dest_path)


/home/alexi/Documents/SBL_MD_CovDock/topology/Dock/APO_6T3D_ADT_rigid.pdbqt prod_structures/Ligand_Substrate/PDBQT_Flex/APO_6T3D_ADT_rigid.pdbqt
/home/alexi/Documents/SBL_MD_CovDock/topology/Dock/APO_5UL8_ADT_rigid.pdbqt prod_structures/Ligand_Substrate/PDBQT_Flex/APO_5UL8_ADT_rigid.pdbqt
/home/alexi/Documents/SBL_MD_CovDock/topology/Dock/APO_1K55_ADT_rigid.pdbqt prod_structures/Ligand_Substrate/PDBQT_Flex/APO_1K55_ADT_rigid.pdbqt


In [ ]:
pythonsh= "/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/bin/pythonsh"
print(flex_out)

# For the Borons
modified_param_line = "parameter_file "+ os.path.join("..","..","..",settings.config, 
                                                        settings.parent, 
                                                        "AD4.1_bound.dat") + "\n"

for pdb in structures.PDBID:

    for lig in os.listdir(flex_out):
        if "_flex.pdbqt" not in lig:
            continue
        # break
        print(pdb, lig)


        pdb = "6T3D"
        
        # break
        print(os.getcwd())
        script_name = "prepare_gpf4.py"
        prepare_grid_path = os.path.join("..","..","..",os.path.join(settings.config, 
                                                                     settings.parent, 
                                                                     script_name))

        pdb_name = "APO_" +pdb + "_ADT_rigid" + ".pdb"
        pdbqt_name = pdb_name.replace(".pdb", ".pdbqt") #.replace("_flex", "_rigid")
        print(pdbqt_name)
        ligand_name = lig
        
        lig = lig.replace(".pdbqt", "")



        prep_grid_command = [pythonsh,
                            prepare_grid_path,
                            "-r", pdbqt_name,
                            "-l", ligand_name,
                            "-x", ligand_name,
                            "-o", lig + ".gpf", 
                            # "-p", "ligand_types='H,HD,HS,C,A,N,NA,NS,OA,OS,F,Mg,Mn,Zn,Ca,Fe,Cl,Br,I,SA,S,Se,P,GA,GC,GN,GO,GP,G'",
                            
                            "-v"]
        
        # print(" ".join(prep_grid_command))
        dir = "/home/alexi/Documents/SBL_MD_CovDock/topology/Dock"
        try:
            subprocess.run(prep_grid_command, cwd=flex_out)
            with open(os.path.join(flex_out, lig + ".gpf"), "r+") as f:
                lines = f.readlines()

                new_lines = []

                new_lines.append(modified_param_line)
                for line in lines:
                    new_lines.append(line)

                # Write modified lines back to the file
                f.seek(0)
                f.writelines(new_lines)
                f.truncate()            
        except:
            pass
    # break
    # break
# pythonsh= "/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/bin/pythonsh"



        subprocess.run(["touch", "empty"], cwd=flex_out, check=True)

# for lig in os.listdir(flex_out):
#     for pdb in structures.PDBID:
        pdb = "6T3D"
        print(os.getcwd())
        script_name = "prepare_dpf4.py"
        prepare_dock_path = os.path.join("..","..","..",os.path.join(settings.config, 
                                                                     settings.parent, 
                                                                     script_name))

        prep_dock_command = [pythonsh,
                            prepare_dock_path,
                            "-l", ligand_name,
                            "-x", ligand_name,
                            "-r", pdbqt_name,
                            "-o", lig+ ".dpf" ,
                            "-p", "move='empty'",
                            # "-p", "rmsdatoms='all'"
                            "-v"]

        # print(" ".join(prep_dock_command))
        dir = "/home/alexi/Documents/SBL_MD_CovDock/topology/Dock"

        # subprocess.run(prep_dock_command, cwd=flex_out, check=True)



        try:
            print(" ".join(prep_dock_command))
            subprocess.run(prep_dock_command, cwd=flex_out, check=True)
            
            with open(os.path.join(flex_out, lig + ".dpf"), "r+") as f:
                lines = f.readlines()

                new_lines = []

                new_lines.append(modified_param_line)                                                      
                for line in lines:
                    if line.startswith("unbound_model"):
                        line = "unbound_energy 0.0      # COVALENT " + "\n"
                    new_lines.append(line)

                new_lines.append("rmsdatoms all     # COVALENT "+ "\n")

                # Write modified lines back to the file
                f.seek(0)
                f.writelines(new_lines)
                f.truncate()
        except:
            pass

    break


In [68]:
# print number of dpf and gpf files
print(len(os.listdir(flex_out)))

print(len([x for x in os.listdir(flex_out) if x.endswith(".dpf")]))
print(len([x for x in os.listdir(flex_out) if x.endswith(".gpf")]))

505
167
167


In [69]:
### https://www.researchgate.net/post/What_are_the_potential_shell_well_and_atomic_solvation_parameters_for_the_boron_atom

In [70]:
# 5. Run AutoGrid and AutoDock



autodockgpu_dir = "/home/alexi/Documents/AutoDock-GPU/bin"
engine = "CUDA_64"

autodockgpu = f"autodock_{engine}wi"
auto_dock = os.path.join(autodockgpu_dir, autodockgpu)
docked = []
# for pdb in structures.PDBID:
pdb="6T3D"
for lig in os.listdir(flex_out):
    if "_flex.dpf" not in lig:
        continue
    lig = lig.replace(".dpf", "")
    print(lig)

# for pdb in structures.PDBID:
    # print("Docking ", lig, " to ", pdb)
    name = "APO_" +pdb + "_ADT"+ "_rigid"
    pdb_name =  name + ".pdb"
    pdbqt_name = pdb_name.replace(".pdb", ".pdbqt")

    grid_name = lig +".gpf"
    grid_log = grid_name.replace(".gpf", ".glg")
    grid_log = f"{lig}_{name}.glg"


    autogrid_command = ["autogrid4", 
                        "-p", grid_name, 
                        "-l", grid_log, 
                        # "-d", f"{lig}_{name}.maps.fld",
                        # "-o", f"{lig}_{name}.maps.map",
    ]
                        # "-v"]

    dock_name = lig + ".dpf"
    dock_log = dock_name.replace(".dpf", ".dlg")

    autodock_command = [auto_dock, 
                        # "-p", dock_name, 
                        # "-l", dock_log, 
                        "-M",  f"{name}.maps.fld", 
                        "-L", f"{lig}.pdbqt",
                        "--lsmet", "ad",
                        # "--psize", "50",
                        # "--nrun", "50",
                        "-v"]


    dir = "/home/alexi/Documents/SBL_MD_CovDock/topology/Dock"
    # print(os.listdir(dir))
    try:
        subprocess.run(autogrid_command, check=True,cwd=flex_out)
        # subprocess.run(autodock_command, check=True,cwd=flex_out)        
        # check that the map fld contains ligand - otherwise command does not run before file is printed
        with open(os.path.join(flex_out, f"{name}.maps.fld"), "r") as f:
            lines = f.readlines()
            for line in lines:
                if grid_name in line:
                    print(line)
                    print("Ligand in map fld")
                    # break
                    subprocess.run(autodock_command, check=True,cwd=flex_out)        
                    print("Docking successful")
                    docked.append((lig, pdb))
    except:
        pass
    # break
print(docked)

6T3D_SER_1X6_flex
#GRID_PARAMETER_FILE 6T3D_SER_1X6_flex.gpf

Ligand in map fld
AutoDock-GPU version: v1.5.3-53-gf58a5173fa2f4c82a74b40bc895ad7e65de59f47

Running 1 docking calculation

Cuda device:                              NVIDIA GeForce RTX 3060
Available memory on device:               11416 MB (total: 12022 MB)

CUDA Setup time 0.051461s

Running Job #1
    Using heuristics: (capped) number of evaluations set to 1132076
    Local-search chosen method is: ADADELTA (ad)

Rest of Setup time 0.003024s

Executing docking runs, stopping automatically after either reaching 0.15 kcal/mol standard deviation of
the best molecules of the last 4 * 5 generations, 42000 generations, or 1132076 evaluations:

Generations |  Evaluations |     Threshold    |  Average energy of best 10%  | Samples |    Best energy
------------+--------------+------------------+------------------------------+---------+-------------------
          0 |          150 |   30.74 kcal/mol |    8.39 +/-    5.96 kcal/mol 


autogrid4: ERROR:  unknown ligand atom type Ru
add parameters for it to the parameter library first!




6T3D_SER_RM9_flex
#GRID_PARAMETER_FILE 6T3D_SER_RM9_flex.gpf

Ligand in map fld
AutoDock-GPU version: v1.5.3-53-gf58a5173fa2f4c82a74b40bc895ad7e65de59f47

Running 1 docking calculation

Cuda device:                              NVIDIA GeForce RTX 3060
Available memory on device:               11416 MB (total: 12022 MB)

CUDA Setup time 0.061226s

Running Job #1
    Using heuristics: (capped) number of evaluations set to 1132076
    Local-search chosen method is: ADADELTA (ad)

Rest of Setup time 0.002792s

Executing docking runs, stopping automatically after either reaching 0.15 kcal/mol standard deviation of
the best molecules of the last 4 * 5 generations, 42000 generations, or 1132076 evaluations:

Generations |  Evaluations |     Threshold    |  Average energy of best 10%  | Samples |    Best energy
------------+--------------+------------------+------------------------------+---------+-------------------
          0 |          150 |17689.00 kcal/mol | 9033.13 +/- 2308.23 kcal/mol 

In [ ]:
print(os.getcwd())

In [ ]:
import os

def get_directory_details(directory_path):
    total_size = 0
    file_count = 0
    
    for dirpath, dirnames, filenames in os.walk(directory_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):  # skip symbolic links
                total_size += os.path.getsize(fp)
            file_count += 1

    return file_count, total_size

directory_path = 'start_structures'
file_count, total_size = get_directory_details(directory_path)
print(f"Number of files: {file_count}")
print(f"Total size: {total_size} bytes")
print(f"Total size: {total_size / 1024**2} MB")